In [1]:
import pandas as pd
import os
from glob import glob

In [2]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda:0")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3060


In [ ]:
# read corpora
ambitious = pd.read_csv('../corpora/ambitious_goals.csv')
confidence = pd.read_csv('../corpora/confidence_in_goals.csv')
contrast = pd.read_csv('../corpora/contrast.csv')
lists = glob('raw/Lists*')
similie = glob('raw/Metaphor*')
rhetoricalq = glob('raw/Rhe*')
sentiment = glob('raw/Sentiment*')
story = glob('raw/Story*')
vua_metaphor = pd.read_csv('../corpora/vua_metaphor.csv')
vua_metaphor = vua_metaphor[['sentence', 'label']].rename(columns={'label':'Metaphor'})
vua_metaphor